# Abstract 

La posibilidad que brindan los sistemas automatizados de decision y clasificacion son sobresalientes al producir mejoras de producto y reducir costos o aumentar ganacias, de manera sistematica y eficaz. Muchas empresas relacionadas al juego y apuestas utilizan esto para predecir numeros ante la gran cantidad de datos que se nos brinda dados en el dataset. Asi mismo , el cliente propone una solucion practica ante las circunstacias economicas que atraviesa.

En base a esta informacion , se puede hacer una investigacion y resolver las practicas beneficiosas de los jugadores y del equipo en cada uno de los juegos, es decir , es posible hacer preguntas y responderlas para obtener nuevos conocimientos sobre los equipos, los jugadores y la competencia en sí. Por ejemplo, se puede analizar la tendencia de los equipos en casa y fuera de casa, las estadísticas de los jugadores en diferentes momentos del partido, o el porcentaje de victorias y derrotas de los equipos.

Este proyecto se basara en utilizar los datos disponibles para encontrar las diferencias y probrabilidades en cada partido dado durante los años jugados por la NBA. El objetivo se centrara en poder adquirir una variable que determine un numero en el cual se basara la cuota cobrada por el sistema de apuestas. Ademas , este analisis proovera al cliente con el gran beneficio del conocimiento y la ventaja por sobre la competencia , o hacia los mismos usuarios de la plataforma.

Asi, lo visto durante un analisis exploratorio de los datos y las caracteristicas del conjunto , se plantearan las primeras hipotesis para luego establecer un modelo predictivo de los datos de interes. Luego de un resultado final , se podra verificar el uso de las herramientas y las capacidades un sistema de ML.

# Entrenando un algoritmo de Machine Learning (Segunda parte)

Una de las bases del entrenamiento de un modelo se basa en las variables que se utilizaran para el mimso , la correlacion con la *variable Target* y la variabilidad de estas , seran determinantes a la hora de elegirlas.
Las variable existentes no muestran de manera directa la relacion de que **un equipo gane segun su comportamiento en temporadas pasadas/actual** . En estos casos , es sugerible realizar nuevas variables en el propio dataset que brinden nueva informacion y esta informacion demuestre la correlacion con las variable de interes

## Desarrollo del codigo

In [ ]:
#Cargar librerias principales
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings
# Suprimir las advertencias para futuros usos
warnings.filterwarnings("ignore")

## Feature Engenering: WIN_SEASONS & WIN_SEASON_C

Para poder establecer que equipos poseen un mejor rendimiento o que poseen una mayor posibilidad de ganar , se decidio basarse en las cantidad de veces que gano y la cantidad de veces que gano consecutivamente (como se derrollo en la ultima entrega):

**WIN_SEASONS** : Cantidad de temporadas ganadas del equipo

**WIN_SEASONS_C** : Cantidad de temporadas ganadas consecutivas del equipo (Veces que sucedio)

Estas variables almacenaran los valores que se obtubieron en insights ya presentados

Hay que recordar que en analisis anterior se concluyo que:

Los puntajes en los equipos durante las temporadas es similar

Los equipos que ganaron alguna ves el campeonato tienen una mayor probabilidad de hacerlo nuevamente con respecto al azar

Los equipos que ganaron el campeonato actualmente , tendran una mayor probabilidad de hacerlo nuevamente el proximo año


In [ ]:
# Se cargara el dataset de entrenamiento (el ds origianl = clean_games.csv) 
# y actualizara con los valores que queremos agregar (team_wins.csv)
os.chdir('D:/Documents/GitHub/NBA_games_analitycs/data')
ds_path='clean_games.csv'
df = pd.read_csv(ds_path)
tw_path='team_wins.csv'
tw = pd.read_csv(tw_path)

# Define la codificación alfabética
equipos = {}
equipo_actual = 'A'

# Itera sobre las filas del DataFrame y realiza los cambios en las columnas HOME_TEAM_ID y VISITOR_TEAM_ID
for index, row in df.iterrows():
    home_team_id = row['TEAM_ID_home']
    visitor_team_id = row['TEAM_ID_away']
    
    # Verifica si el equipo ya está codificado, si no, le asigna una nueva letra
    if home_team_id not in equipos:
        equipos[home_team_id] = equipo_actual
        equipo_actual = chr(ord(equipo_actual) + 1)
    
    if visitor_team_id not in equipos:
        equipos[visitor_team_id] = equipo_actual
        equipo_actual = chr(ord(equipo_actual) + 1)
    
    # Actualiza los valores de los nombres de los equipos en las columnas correspondientes
    df.at[index, 'TEAM_ID_home'] = equipos[home_team_id]
    df.at[index, 'TEAM_ID_away'] = equipos[visitor_team_id]
#df.head()

In [ ]:
tw

In [ ]:
# Previo a continuar , modificaremos los nombres de las columnas para que no generen conflicto en los dataframe a usar
tw = tw.rename(columns={'TEAM_ID': 'WIN_SEASONS','veces_consecutivas':'WIN_SEASONS_C'})
# Crearemos un DataFrame para almacenar los valores acumulados por equipo y temporada
equipos = tw["WIN_SEASONS"].unique()
temporadas = tw["SEASON"].unique()
register = pd.DataFrame(columns=equipos, index=temporadas)
# Calcular los valores acumulados por equipo y temporada
for temporada in temporadas:
    for equipo in equipos:
        df_temp = tw[(tw["SEASON"] <= temporada) & (tw["WIN_SEASONS"] == equipo)]
        register.loc[temporada, equipo] = df_temp["WIN_SEASONS"].apply(len).sum()

# Se agregan al DataFrame los datos
df = pd.merge(df,tw[['SEASON', 'WIN_SEASONS']], on='SEASON', how='left')
df = pd.merge(df,tw[['SEASON', 'WIN_SEASONS_C']], on='SEASON', how='left')

df

In [ ]:
# Crear un DataFrame para almacenar los valores acumulados por equipo y temporada
equipos = tw["WIN_SEASONS"].unique()
temporadas = tw["SEASON"].unique()
register = pd.DataFrame(columns=equipos, index=temporadas)

# Calcular los valores acumulados por equipo y temporada
for temporada in temporadas:
    for equipo in equipos:
        df_temp = tw[(tw["SEASON"] <= temporada) & (tw["WIN_SEASONS"] == equipo)]
        register.loc[temporada, equipo] = df_temp["WIN_SEASONS"].apply(len).sum()

print(register)

In [ ]:
df.head()

In [ ]:
def obtener_valor_register(season,home_id,register):
    if home_id in register.columns:
        temp = register[home_id]
        temp_2 = temp[season]
    else:
        temp_2 = 0
    return temp_2


for match in range(len(df['WIN_SEASONS'])):
    season=int(df.iloc[match,df.columns.get_indexer(['SEASON'])].values)
    team=df.iloc[match ,df.columns.get_indexer(['TEAM_ID_home'])].values.item(0)
    temp = obtener_valor_register(season,team,register)
    df['WIN_SEASONS'][match] = temp

df
#7 min de ejecucion

Ya definidas las variables de interes , se procedera a realizar un modelo que realice la prediccion de un valor de importancia (o peso) , es decir , se **realizara una variable muestre un factor de "ganador"**

**WIN_FACTOR** : Factor que muestra posibilidad de ganar del equipo

Lo que se intenta con esta variable es poder modificar la variable target durante el primer cuarto de un partido, para que el modelo tenga una mayor precision al inicio del mismo , ya que al tener pocos datos (datos que se recopilan en tiempo real) el modelo podria estar dando un valor de cuota erroneo totalmente en los primeros 12 minutos.

Para estimar el valor de una variable acorde de probabilidad (entre 0 y 1) , se usara el anterior desarrollo de proporciones de las situaciones a favor en las que ganaron los equipos:

In [ ]:
# Cantidad de equipos que participan en la temporada para estimar una probabilidad clasica de que cada equipo sea victorioso
n_equipos = 60
p_equipos = 1/60
print(f'La probabilidad clasica de ganar es de :',round(p_equipos,2))

# Cantidad de veces que hubo victorias consecutivas, temporadas en las que se produjo un caso y la proporcion de casos
ganadas_consecutivas = 5
temporadas_consecutivas = 19
equipos_consecutivas = 4
proporcion_consecutivas = ganadas_consecutivas/(temporadas_consecutivas*equipos_consecutivas)
print(f'La proporcion de ganadas consecutivas por equipo es de :',round(proporcion_consecutivas,2))

# Cantidad de veces que hubo victorias individuales, temporadas en las que se pudo producir un caso y la proporcion de casos
ganadas_individuales = 13 # No consecutivas
equipos_individuales = 7  # Equipos que ganaron una temporada sin ser una consecutiva
temporadas_individuales = 20
proporcion_individuales = ganadas_individuales/(temporadas_individuales*equipos_individuales)
print(f'La proporcion de ganadas individuales por equipo es de :',round(proporcion_individuales,2))

Asi  podemos estimar de manera discreta , que la posibilidad de ganar de un equipo que gano al menos una ves es del 7% y la probabilidad de ser victorioso porque gano la temporada anterior es de un 9%
Para el ultimo caso , la probabilidad es condicional debido a que es necesario que gane la temporada actual (y eso tiene una probabilidad asociada).Asi , se tomaran 3 casos para estimar las bases:

**Si el equipo no gano ninguna temporada hasta ahora , se considerara como una situacion normal en la que tiene la misma posibilidad de ganar que los demas , es decir, 2%**

**Si el equipo gano al menos una temporada hasta ahora, se considerara una probabilidad mayor a la situacion normal , que deberia ser la que el equipo es de mejor rendimiento que la mayoria , es decir , la probabilidad estadistica + la probabilidad por ser un "equipo de mejor rendimiento" , 2% + 7% = 9%**

**Si el equipo gano la ultima temporada, se considera que el equipo es de mayor rendimiento que la mayoria y que existe la posibilidad de continuar la racha por "un mejor estado del equipo" demostrado en la ultima temporada: 2% + 7% + 9% = 18%**

Teniendo las bases para las condiciones , se puede establecer la ecuacion con los parametros ajustados a un valor apropiado:

PC= Probabilidad estadistica : [1 equipo]/[N de equipos]

PGC = Probabilidad por estado : [Victorias consecutivas de equipos]/[Temporada consecutivas de equipos * N de equipos que ganaron consecutivamente]

PGI = Probabilidad por rendimiento y club : [Victorias individuales de equipos]/[Temporada individuales de equipos * N de equipos que ganaron al menos una ves]

WF = PC + PGC + PGI 

Como estimacion por puntaje , presenta un valor aceptable pero el tiempo es parte del efecto de estas posibilidades, por lo tanto , sera necesario tomar algunos parametros extra para ajustar aun mejor la variable.

WF = PC + PGC x UT + PGI x RR

UT (Ultima Temporada) = Parametro binario que determina si gano o no la ultima temporada : [ 1 | 0 ]

RR (Rendimiento Reciente) = Parametro que determina en que medida se puede estimar el rendimiento de un equipo debido a las ganadas de las temporadas anteriores : [Numero de temporada victoriosas de cada equipo] / [Numero de temporada totales que participo el equipo]

De esta manera es posible parametrizar el peso de cada situacion que genera una adicion de probabilidad a ganar

In [ ]:
def obtener_win_factor(season,home_id,register):
    if home_id in register.columns:
        temp = register[home_id]
        temp_2 = temp[season]
    else:
        temp_2 = 0
    return temp_2


for match in range(len(df['WIN_SEASONS'])):
    season=int(df.iloc[match,df.columns.get_indexer(['SEASON'])].values)
    team=df.iloc[match ,df.columns.get_indexer(['TEAM_ID_home'])].values.item(0)
    temp = obtener_win_factor(season,team,register)
    df['WIN_SEASONS'][match] = temp

df

A partir de estas variables agregadas es posible realizar un modelo que las relacione y realice una prediccion para cada partido.
Se procedera a armar el dataset con las variables y organizarlo correctamente para que el modelo pueda ser entrenado.